In [27]:
import math
import numpy as np
import seaborn as sns

C:\Users\arend\anaconda3\envs\zeearend\envs\zeearend\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [4]:
mandelbrot_set = np.array([1j+ 1, 2j, 3j])

In [6]:
np.imag(mandelbrot_set)
np.real(mandelbrot_set)

array([1., 0., 0.])

In [45]:
def mandelbrot_iteration(z, c):
    return z**2 + c

def mandelbrot(c, num_iterations):
    z = 0
    z_list = np.array([z])
    for _ in range(num_iterations):
        z = z**2 + c
        if z in z_list:
            return True
        z_list = np.append(z_list, z)
    return False

In [17]:
def complex_matrix(xmin, xmax, ymin, ymax, pixel_density):
    re = np.linspace(xmin, xmax, int((xmax - xmin) * pixel_density))
    im = np.linspace(ymin, ymax, int((ymax - ymin) * pixel_density))
    return re[np.newaxis, :] + im[:, np.newaxis] * 1j

In [54]:
mat = complex_matrix(-2, 2, -2, 2, 100)

In [ ]:
for i in range(np.shape(mat)[0]):
    for j in range(np.shape(mat)[1]):
        v = mandelbrot(mat[i][j], 100)
        mat[i][j] = v

<ipython-input-45-31573ef9bf53>:8: RuntimeWarning: overflow encountered in cdouble_scalars
  z = z**2 + c
<ipython-input-45-31573ef9bf53>:8: RuntimeWarning: invalid value encountered in cdouble_scalars
  z = z**2 + c


In [ ]:
print(y)
print(n)

In [ ]:
sns.heatmap(np.real(mat))